In [1]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
fm = pd.read_csv('newest_fm.csv')
fm_clean = fm.drop_duplicates(subset=['book_QN'])
display(fm)

,Unnamed: 0,id,title,language,downloads,book_QN,author_QN,authorLabel
0,0,PG10002,The House on the Borderland,['en'],824,Q740473,Q919277,William Hope Hodgson
1,1,PG10005,A Voyage to the Moon: With Some Account of the...,['en'],42,Q20543676,Q1508382,George Tucker
2,2,PG10008,The Mystery,['en'],27,Q95981569,Q185406,Samuel Hopkins Adams
3,3,PG10046,Salute to Adventurers,['en'],24,Q7406333,Q333004,John Buchan
4,4,PG10055,Hamburgische Dramaturgie,['de'],57,Q53585,Q34628,Gotthold Ephraim Lessing
...,...,...,...,...,...,...,...,...
4550,866,PG9070,The Imaginary Invalid,['en'],1057,Q588161,Q687,Molière
4551,872,PG944,The Voyage of the Beagle,['en'],874,Q1768515,Q1035,Charles Darwin
4552,884,PG9749,The Highwayman,['en'],29,Q11153710,Q1390604,H. C.Bailey
4553,885,PG9828,Barnen ifrån Frostmofjället,['sv'],36,Q10426505,Q4948162,Laura Fitinghoff


In [3]:
def getresult(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0")
#     print("connected")
    sparql.addExtraURITag("timeout","10000")
    sparql.setQuery(query)
#     print("A")
    sparql.setReturnFormat(JSON)
#     print("B")
#     try:
    results = sparql.queryAndConvert()
#     print("C")
#     except Exception as e:
#         print(e)
    results = results['results']['bindings']
    if results:
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results

In [ ]:
# query = """
# SELECT ?work ?author ?workLabel (Year(?publicationDate) AS ?PublicationYear) ?authorLabel ?genderLabel ?citizenshipLabel ?birthYear ?deathYear ?religionLabel ?nativeLanguageLabel
# WHERE {
#   VALUES ?work {wd:Q43361}
#   ?work wdt:P577 ?publicationDate.
#   VALUES ?author {wd:Q34660}
#   ?author   wdt:P21 ?gender;          
#             wdt:P27 ?citizenship;      
#             wdt:P569 ?birthDate;
#   OPTIONAL { ?author wdt:P570 ?deathDate.}
#   OPTIONAL { ?author wdt:P140 ?religion. }
#   OPTIONAL { ?author wdt:P103 ?nativeLanguage. }
#   BIND(YEAR(?birthDate) AS ?birthYear)
#   BIND(YEAR(?deathDate) AS ?deathYear)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
# }
# """
# p = getresult(query)
# result_df = pd.DataFrame.from_dict(p)
# display(result_df)

In [ ]:
# base_query = """
# SELECT ?work ?author ?workLabel (Year(?publicationDate) AS ?PublicationYear) ?authorLabel ?genderLabel ?citizenshipLabel ?birthYear ?deathYear ?religionLabel ?nativeLanguageLabel
# WHERE {
#   VALUES ?work {wd:%s}
#   ?work wdt:P577 ?publicationDate.
#   VALUES ?author {wd:%s}
#   ?author   wdt:P21 ?gender;          
#             wdt:P27 ?citizenship;      
#             wdt:P569 ?birthDate;
#   OPTIONAL { ?author wdt:P570 ?deathDate.}
#   OPTIONAL { ?author wdt:P140 ?religion. }
#   OPTIONAL { ?author wdt:P103 ?nativeLanguage. }
#   BIND(YEAR(?birthDate) AS ?birthYear)
#   BIND(YEAR(?deathDate) AS ?deathYear)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
# }
# """
# for index, row in fm.iterrows():
#     book = row['book_QN']
#     author = row['author_QN']
#     formatted_query = base_query % (book, author)
#     df_1 = getresult(formatted_query)
#     if df_1 is not None:
#         rdf = pd.DataFrame.from_dict(df_1)
#         result_df = pd.concat([result_df,rdf])
# display(result_df)

In [4]:
query = """
SELECT ?work ?workLabel (Year(?publicationDate) AS ?PublicationYear) ?genre ?genreLabel
WHERE {
  VALUES ?work {wd:Q43361}
  ?work wdt:P577 ?publicationDate;
  OPTIONAL { ?work wdt:P136 ?genre. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
p = getresult(query)
result_book = pd.DataFrame.from_dict(p)
display(result_book)

,work,genre,workLabel,genreLabel,PublicationYear
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q132311,Harry Potter and the Philosopher's Stone,fantasy,1997
1,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q21802675,Harry Potter and the Philosopher's Stone,adventure fiction,1997
2,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q111984153,Harry Potter and the Philosopher's Stone,young adult fiction,1997


In [5]:
book_query = """
SELECT ?work ?workLabel (Year(?publicationDate) AS ?PublicationYear) ?genre ?genreLabel
WHERE {
  VALUES ?work {wd:%s}
  ?work wdt:P577 ?publicationDate;
  OPTIONAL { ?work wdt:P136 ?genre. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
for index, row in fm.iterrows():
    book = row['book_QN']
    formatted_query = book_query % (book)
    book_df_1 = getresult(formatted_query)
    if book_df_1 is not None:
        book_df = pd.DataFrame.from_dict(book_df_1)
        result_book = pd.concat([result_book,book_df])
display(result_book)

,work,genre,workLabel,genreLabel,PublicationYear
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q132311,Harry Potter and the Philosopher's Stone,fantasy,1997
1,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q21802675,Harry Potter and the Philosopher's Stone,adventure fiction,1997
2,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q111984153,Harry Potter and the Philosopher's Stone,young adult fiction,1997
0,http://www.wikidata.org/entity/Q740473,http://www.wikidata.org/entity/Q132311,The House on the Borderland,fantasy,1908
1,http://www.wikidata.org/entity/Q740473,http://www.wikidata.org/entity/Q16575965,The House on the Borderland,horror fiction,1908
...,...,...,...,...,...
0,http://www.wikidata.org/entity/Q1768515,http://www.wikidata.org/entity/Q1164267,The Voyage of the Beagle,travel book,1839
1,http://www.wikidata.org/entity/Q1768515,http://www.wikidata.org/entity/Q3337126,The Voyage of the Beagle,nature writing,1839
0,http://www.wikidata.org/entity/Q11153710,http://www.wikidata.org/entity/Q474090,The Highwayman,narrative poetry,1906
0,http://www.wikidata.org/entity/Q10426505,NaN,Barnen ifrån Frostmofjället,NaN,1907


In [6]:
result_book.to_csv('book_info.csv')

In [7]:
query = """
SELECT ?author ?authorLabel ?genderLabel ?citizenshipLabel ?birthYear ?deathYear ?religionLabel ?nativeLanguageLabel
WHERE {
  VALUES ?author {wd:Q34660}
  ?author   wdt:P21 ?gender;              
            wdt:P569 ?birthDate;
  OPTIONAL { ?author wdt:P570 ?deathDate.}
  OPTIONAL { ?author wdt:P27 ?citizenship.}
  OPTIONAL { ?author wdt:P140 ?religion. }
  OPTIONAL { ?author wdt:P103 ?nativeLanguage. }
  BIND(YEAR(?birthDate) AS ?birthYear)
  BIND(YEAR(?deathDate) AS ?deathYear)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
p = getresult(query)
result_author = pd.DataFrame.from_dict(p)
display(result_author)

,author,birthYear,authorLabel,genderLabel,citizenshipLabel,religionLabel,nativeLanguageLabel
0,http://www.wikidata.org/entity/Q34660,1965,J. K. Rowling,female,United Kingdom,Anglicanism,British English


In [8]:
authors_only = fm.drop_duplicates(subset=['author_QN'])
author_query = """
SELECT ?author ?authorLabel ?genderLabel ?citizenshipLabel ?birthYear ?deathYear ?religionLabel ?nativeLanguageLabel
WHERE {
  VALUES ?author {wd:%s}
  ?author   wdt:P21 ?gender;          
            wdt:P27 ?citizenship;      
            wdt:P569 ?birthDate;
  OPTIONAL { ?author wdt:P570 ?deathDate.}
  OPTIONAL { ?author wdt:P140 ?religion. }
  OPTIONAL { ?author wdt:P103 ?nativeLanguage. }
  BIND(YEAR(?birthDate) AS ?birthYear)
  BIND(YEAR(?deathDate) AS ?deathYear)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
for index, row in authors_only.iterrows():
    author = row['author_QN']
    print(author)
    formatted_query = author_query % (author)
    author_df_1 = getresult(formatted_query)
    if author_df_1 is not None:
        author_df = pd.DataFrame.from_dict(author_df_1)
        result_author = pd.concat([result_author,author_df])
display(result_author)

Q919277
Q1508382
Q185406
Q333004
Q34628
Q207544
Q468156
Q42511
Q525713
Q167856
Q36184
Q7167372
Q30875
Q477999
Q6759691
Q5686
Q1387643
Q1564825
Q146763
Q127332
Q20733
Q45765
Q170509
Q704860
Q33977
Q271961
Q212719
Q5879
Q1294394
Q353238
Q1444002
Q453577
Q548040
Q2668090
Q50020
Q535
Q219829
Q7245
Q692
Q348623
Q151403
Q438241
Q234530
Q35610
Q82925
Q36322
Q16867
Q314158
Q502912
Q148234
Q25351
Q504
Q1371577
Q1676368
Q9327
Q202749
Q131333
Q16063166
Q132805
Q259132
Q458658
Q41166
Q43718
Q313932
Q771698
Q237196
Q507991
Q462363
Q44519
Q763341
Q447240
Q40826
Q893138
Q478450
Q154211
Q944063
Q311854
Q47484
Q9711
Q276032
Q42443
Q126596
Q460366
Q2065950
Q5337523
Q5075251
Q102513
Q1699846
Q18911708
Q5531480
Q506757
Q2517956
Q965927
Q1382113
Q713284
Q276028
Q4798900
Q457287
Q34190
Q183167
Q3297881
Q560015
Q35064
Q191026
Q123469
Q5536746
Q6221558
Q369790
Q984
Q246953
Q723374
Q318399
Q37693
Q442130
Q41502
Q115822
Q33866
Q326138
Q216904
Q510258
Q38337
Q258741
Q82006
Q3611689
Q1131497
Q81796
Q3889326
Q3668

,author,birthYear,authorLabel,genderLabel,citizenshipLabel,religionLabel,nativeLanguageLabel,deathYear
0,http://www.wikidata.org/entity/Q34660,1965,J. K. Rowling,female,United Kingdom,Anglicanism,British English,NaN
0,http://www.wikidata.org/entity/Q919277,1877,William Hope Hodgson,male,United Kingdom of Great Britain and Ireland,NaN,NaN,1918
1,http://www.wikidata.org/entity/Q919277,1877,William Hope Hodgson,male,United Kingdom of Great Britain and Ireland,NaN,NaN,1918
0,http://www.wikidata.org/entity/Q1508382,1775,George Tucker,male,United States of America,NaN,NaN,1861
0,http://www.wikidata.org/entity/Q185406,1871,Samuel Hopkins Adams,male,United States of America,NaN,NaN,1958
...,...,...,...,...,...,...,...,...
0,http://www.wikidata.org/entity/Q312407,1896,John Dos Passos,male,United States of America,NaN,NaN,1970
0,http://www.wikidata.org/entity/Q1151,1803,Hector Berlioz,male,France,atheism,NaN,1869
0,http://www.wikidata.org/entity/Q3023540,1881,Princess Der Ling,female,Qing Dynasty,Catholic Church,NaN,1944
0,http://www.wikidata.org/entity/Q1390604,1880,Alfred Noyes,male,United Kingdom,Catholic Church,NaN,1958


In [9]:
result_author.to_csv('author_info.csv')

In [ ]:
# result_df = result_df.drop(result_df.index[0], inplace=True)
# result_df = result_df.drop_duplicates(subset=['work','author'])
copy = result_df.copy()
copy = copy.reset_index()
display(copy)

In [ ]:
copy1 = copy[copy['work'] != "http://www.wikidata.org/entity/Q43361"]
copy1 = copy1.drop_duplicates(subset=['work','author'])
copy1 = copy1.rename(columns={'author':"author_y","work":"book"})
exact_match = pd.merge(result, copy1, on = ['book','author_y'])
display(exact_match)
exact_match.to_csv('exact_match.csv')

In [ ]:
em = pd.read_csv('exact_match.csv')
em = em[em['deathYearMatch'] == 1]
em.to_csv("EEM.csv")
display(em)

In [ ]:
query = """
SELECT ?personLabel ?work ?workLabel
WHERE {
  ?person wdt:P800 ?work.
  ?work wdt:P31 ?worktype.
  FILTER(?worktype = wd:Q7725634 || ?worktype = wd:Q1279564 || ?worktype = wd:Q47461344)
  VALUES ?person { wd:Q34660 } # J. K. Rowling's Q number
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
p = getresult(query)
result_author = pd.DataFrame.from_dict(p)
display(result_author)

In [ ]:
author = pd.read_csv('author_info.csv')
author['author_QN'] = author['author'].str.split('/').str[-1]
display(author)

In [ ]:
author_query = """
SELECT ?person ?personLabel ?work ?workLabel
WHERE {
  ?person wdt:P800 ?work.
  ?work wdt:P31 ?worktype.
  FILTER(?worktype = wd:Q7725634 || ?worktype = wd:Q1279564 || ?worktype = wd:Q47461344)
  VALUES ?person { wd:%s } # J. K. Rowling's Q number
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
for index, row in author.iterrows():
    name = row['author_QN']
    formatted_query = author_query % (name)
    author_df_1 = getresult(formatted_query)
    if author_df_1 is not None:
        author_df = pd.DataFrame.from_dict(author_df_1)
        result_author = pd.concat([result_author,author_df])
display(result_author)

In [ ]:
result_author.to_csv('author_to_book.csv')

In [ ]:
result_author['author'] = result_author['person'].str.split('/').str[-1]
result_author['last_name'] = result_author['personLabel'].str.split(' ').str[-1]
full = pd.read_csv('metadata.csv')
full['last_name'] = full['author'].str.split(',').str[0]
matches = pd.merge(full[['title','last_name']],result_author, on = ['last_name'])
matches = matches.drop_duplicates(subset=['title','last_name','workLabel'])
display(matches)

In [ ]:
#display(matches[matches['title'] == matches['workLabel']])
display(matches[(matches['author'] == "Q535") & (matches['title'] == matches['workLabel'])])

In [ ]:
from fuzzywuzzy import fuzz
match_score = []
for index, row in matches.iterrows():
    title = row['title']
    work = row['workLabel']
    score = fuzz.ratio(title, work)
    match_score.append(score)
matches['match_score'] = match_score
display(matches[(matches['match_score'] >= 80) & matches['match_score'] != 100])

In [ ]:
display(matches[(matches['match_score'] >= 80) & (matches['match_score'] != 100)])

In [ ]:
matches.to_csv("match_scores.csv")